# 0. Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

## 5. Data Modeling
**Bài toán**: Dự đoán một công ty đã nằm trong bảng xếp hạng FT1000 có tiếp tục nằm trong bảng xếp hạng FT1000 năm sau hay không?

**Ý nghĩa**: Cùng ý nghĩa với các câu hỏi ở phần Data Analyzing, khi biết được một công ty đã được xếp hạng trong FT1000 năm nay có tiếp tục nằm trong bảng xếp hạng FT1000 vào năm sau hay không giúp các nhà đầu tư đưa ra lựa chọn đầu tư chính xác hơn, bên cạnh các chỉ số về doanh thu hay mức tăng trưởng đã được phân tích trong bảng xếp hạng năm nay

## 5.1. Thu thập thêm dữ liệu
Dữ liệu bảng xếp hạng năm nay là không đủ để  giải quyết bài toán, ta cần thu thập dữ liệu của cả những bảng xếp hạng trước. Bảng xếp hạng FT tính đến namư 2022 được tổ chức lần thứ 6, do đó ta cần thu thập thêm dữ liệu từ 5 bảng xếp hạng ở 5 năm trước. Phương pháp thu thập cũng sẽ giống việc thu thập ở phần Data Collecting

In [ ]:
url = 'https://ig.ft.com/ft-1000/'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2017 = pd.DataFrame(js[0])
df2017.rename(columns={'Unnamed: 0':'Rank'}, inplace = True)
df2017 = df2017.sort_values(by = 'Rank').reset_index(drop = True)
df2017.head()

,Rank,Company,Country,Sector,RevenueGrowth 2012-5,CAGR2012-5,Revenue2015 (’000 €),Employees2015,Founded
0,1,HelloFresh,Germany,Food & Beverage,"13,159%",409.9%,304952,981.0,2011
1,2,Codewise,Poland,Advertising,"13,052%",408.5%,31613,103.0,2011
2,3,Green IT Das Systemhaus,Germany,Technology,"11,113%",382.2%,24305,80.0,2012
3,4,Brainlabs,United Kingdom,Advertising,"8,218%",336.5%,13608,48.0,2012
4,5,Catapult Enterprises (Propercorn),United Kingdom,Food & Beverage,"7,510%",323.8%,12052,32.0,2011
